In [1]:
import numpy as np

In [2]:
#Parameters
tf=120
sequence = 30
TTP = 0.8 # Train Test Percentage 
K = 5
epoch = 100
early = 10
lstm1 = 64
lstm2 = 64
random = 0 # on=1 off=0
KFold = 1 # on=1 off=0

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os

directory = r"/content/drive/MyDrive/Paper3/Data"

# get a list of all .txt files in the directory
txt_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.txt')]

# print the number of .txt files found
print("Number of txt files:", len(txt_files))

# read the contents of each .txt file and store them in an array
Data = np.zeros((len(txt_files),50000,4))
for file_path in txt_files:
    for i in range(len(txt_files)):
        data = np.loadtxt(txt_files[i], dtype=object)
        data = [line.split(',') for line in data]

        # Convert the resulting list of lists to a NumPy array with the appropriate data types
        data = np.array(data, dtype=np.float32)
        #data = data.astype(np.int32)
        A = np.zeros((50000-data.shape[0], data.shape[1]))
        Data[i] = np.concatenate((data, A))

# # print the contents of the array
# print("File contents:", file_contents)

Number of txt files: 11


In [5]:
if random ==1:
    Frames = np.zeros((frames.shape[0],frames.shape[1],2),dtype='float') 
    A = np.array(range(12))
    random.shuffle(A)
    for i in range(12):
        Frames[i] = frames[A[i]]
    frames = Frames
    
    
if KFold == 1:
    
    Size = round(len(txt_files)*(1-TTP))
    print("Size: ",Size)
    data_zeros = np.zeros((Data.shape[0], Data.shape[1], Data.shape[2]),dtype='float') 
    startpoint = (K-1)*Size
    A = np.array(range(len(txt_files)))
    for i in range(Size):
        data_zeros[i] = Data[startpoint]
        A = np.delete(A, Size*(K-1))
        startpoint = startpoint + 1
    for i in range(len(txt_files)-Size):
        data_zeros[i+Size] = Data[A[i]]
    Data = data_zeros

Size:  2


In [6]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel(r'/content/drive/MyDrive/Paper3/Properties.xlsx', sheet_name='Sheet1')

# Select the values from row 3 and onwards
Properties = df.values[1:]

In [7]:
import numpy as np

# Assume the data array is already loaded into a variable named 'data'

# Find the indices where the values are "N" or "F"
idx_n = np.where(Properties == np.array(["N"], dtype=str))
idx_f = np.where(Properties == np.array(["F"], dtype=str))

# Replace "N" with "1"
Properties[idx_n] = 1

# Replace "F" with "2"
Properties[idx_f] = 2


In [8]:
Properties = np.array(Properties)
Properties = Properties.astype(float)

In [9]:
resulotion = 0.04
Dim = round((Properties[-1,0]-Properties[0,0])/resulotion)
Properties_res = np.zeros((Dim, Properties.shape[1]))
Data_res = np.zeros((len(txt_files), Dim, Data.shape[2]))
t0 = 5
tf = 620.8
t = t0
Data_inter = np.zeros(len(txt_files),dtype="int")
Properties_inter = 0
#for j in range(len(txt_files)):
for j in range(1):
    for i in range(Data[j].shape[0]):
        if Data[j,i,0] >= t0:
            Data_inter[j] = i - 1
            break
for i in range(Dim):
    Properties_res[i] = Properties[Properties_inter]
    Properties_res[i,0] = t
    if t >= Properties[Properties_inter+1,0]:
        Properties_inter += 1
    for j in range(len(txt_files)):
        if Data[j,Data_inter[j]+2,0]==0:
            continue
        Data_res[j,i] = Data[j, Data_inter[j]]
        Data_res[j,i,0] = t
        while True:
            if t >= Data[j, Data_inter[j] + 1,0]:
                Data_inter[j] += 1
            else:
                break
    
    t += resulotion

In [10]:
# Number 1
XBminN1 = 120 #X Body min Near
XBmaxN1 = 160 #X Body max Near

XBminF1 = 150 #X Body min Far
XBmaxF1 = 170 #X Body max Far


# Number 2
XBminN2 = 160 #X Body min Near
XBmaxN2 = 195 #X Body max Near

XBminF2 = 170 #X Body min Far
XBmaxF2 = 190 #X Body max Far


# Number 3
XBminN3 = 195 #X Body min Near
XBmaxN3 = 230 #X Body max Near

XBminF3 = 190 #X Body min Far
XBmaxF3 = 220 #X Body max Far


In [11]:
Model_data = np.zeros((Data_res.shape[0]*Data_res.shape[1],Properties_res.shape[1]))
Model_label = np.zeros((Data_res.shape[0]*Data_res.shape[1]), dtype="O")
inter = 0
for z in range(Data_res.shape[0]):
    for j in range(Data_res.shape[1]):
        if Data_res[z,j,0] == 0:
            continue
        Model_data[inter] = Properties_res[j]
        Model_label[inter] = "No Data"

        P = 0 # number of person 1
        if Properties_res[j,6*P+1] == 1: #if the person is present
            if Properties_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN1<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"
                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if 100<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"

                        
            elif Properties_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF1<Data_res[z,j,2]<XBmaxF1: #Body
                        Model_label[inter] = "Number 1"

                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if 100<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"


        P = 1 # number of person 2
        if Properties_res[j,6*P+1] == 1: #if the person is present
            if Properties_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN2<Data_res[z,j,2]<XBmaxN2: #Body
                        Model_label[inter] = "Number 2"
                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if Properties_res[j,6*P+6] <=0:  #if the person is leaving or entering from left side
                        if 100<Data_res[z,j,2]<180: #Body
                            Model_label[inter] = "Number 2"
                    elif Properties_res[j,6*P+6] >=0:  #if the person is leaving or entering from left side
                        if 180<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 2"
                            

            elif Properties_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF2<Data_res[z,j,2]<XBmaxF2: #Body
                        Model_label[inter] = "Number 2"

                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if Properties_res[j,6*P+6] <=0:  #if the person is leaving or entering from left side
                        if 100<Data_res[z,j,2]<180: #Body
                            Model_label[inter] = "Number 2"
                    elif Properties_res[j,6*P+6] >=0:  #if the person is leaving or entering from left side
                        if 180<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 2"
                            

        P = 2 # number of person 3
        if Properties_res[j,6*P+1] == 1: #if the person is present
            if Properties_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN3<Data_res[z,j,2]<XBmaxN3: #Body
                        Model_label[inter] = "Number 3"
                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if XBminN3<Data_res[z,j,2]<250: #Body
                        Model_label[inter] = "Number 3"

            elif Properties_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF3<Data_res[z,j,2]<XBmaxF3: #Body
                        Model_label[inter] = "Number 3"
                        
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                        if XBminF3<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 3"
        inter +=1

In [12]:
Model_data = np.array(Model_data[0:inter])
Model_label = np.array(Model_label[0:inter])

In [13]:
Model_data.shape

(164061, 19)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler = scaler.fit(Model_data)
Model_data_normal = scaler.transform(Model_data)  #scale the data

Model_data_normal = np.delete(Model_data_normal, [0,1,7,13], axis=1)

In [15]:
def Labell(String):
    if String == "Number 1":
        return 0
    elif String == "Number 2":
        return 1
    elif String == "Number 3":
        return 2

In [16]:
# Model_data_normal_sequence = np.zeros((Model_data_normal.shape[0]-sequence+1,sequence,Model_data_normal.shape[1]),dtype='float')
# for i in range(Model_data_normal_sequence.shape[0]):
#     for j in range(sequence):
#         Model_data_normal_sequence[i,j] = Model_data_normal[i+j]
        
# Model_data_normal_sequence = np.delete(Model_data_normal_sequence, -1, axis = 0)

In [17]:
Model_data_normal_sequence = np.zeros((Model_data_normal.shape[0],sequence,Model_data_normal.shape[1]),dtype='float')
Model_label_sequence = np.zeros((Model_data_normal.shape[0],1),dtype='int')
inter = 0
Persons_inter = np.zeros(len(txt_files))
inter2 = 0
for i in range(Model_data_normal_sequence.shape[0]-sequence):
    if Model_data[i, 0] == 5:
        Persons_inter[inter2] = inter
        inter2 += 1
    if Model_label[i+sequence] != "No Data" and (Model_data[i+sequence, 0] - Model_data[i, 0]) > 0:
        Model_label_sequence[inter] = Labell(Model_label[i+sequence])
        for j in range(sequence):
            Model_data_normal_sequence[inter,j] = Model_data_normal[i+j]
        inter += 1
        
Model_data_normal_sequence = np.array(Model_data_normal_sequence[0:inter])
Model_label_sequence = np.array(Model_label_sequence[0:inter])

In [18]:
Persons_inter

array([     0.,  14980.,  23535.,  38604.,  51505.,  66447.,  80448.,
        94827., 109418., 122581., 135243.])

In [19]:
Model_data_normal_sequence.shape

(150151, 30, 15)

In [20]:
train_data = np.array(Model_data_normal_sequence[int(Persons_inter[Size]):Model_data_normal_sequence.shape[0]])
train_label = np.array(Model_label_sequence[int(Persons_inter[Size]):Model_label_sequence.shape[0]])
test_data = np.array(Model_data_normal_sequence[0:int(Persons_inter[Size])])
test_label = np.array(Model_label_sequence[0:int(Persons_inter[Size])])

In [21]:
print("train_data:",train_data.shape)
print("train_label:",train_label.shape)
print("test_data:",test_data.shape)
print("test_label:",test_label.shape)

train_data: (126616, 30, 15)
train_label: (126616, 1)
test_data: (23535, 30, 15)
test_label: (23535, 1)


In [22]:
from tensorflow.keras.utils import to_categorical

#X_train = train_data.reshape(train_data.shape[0],sequence, -1)
one_hot_encoded_labels_train = to_categorical(train_label)

#X_test = test_data.reshape(test_data.shape[0],sequence, -1)
one_hot_encoded_labels_test = to_categorical(test_label)

In [23]:
import tensorflow as tf
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = tf.keras.layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

In [24]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = tf.keras.Sequential(
            [tf.keras.layers.Dense(dense_dim, activation='swish'),tf.keras.layers.Dense(embed_dim, activation='swish')]
        )
        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [25]:
def get_compiled_model():
    MAX_SEQ_LENGTH = sequence
    NUM_FEATURES = train_data.shape[2]
    classes = one_hot_encoded_labels_train.shape[1]
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 1024
    num_heads = 2
    #classes = len(set(Label))
    
    inputs = tf.keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, NUM_FEATURES, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(NUM_FEATURES, dense_dim, num_heads, name="transformer_layer")(x)
    x = tf.keras.layers.Add()([inputs,x])
    x = tf.keras.layers.GlobalMaxPooling1D()(x)
#    x = tf.keras.layers.Dropout(0.2)(x)
#    x=tf.keras.layers.Dense(16,activation='swish')(x)
#     x=tf.keras.layers.Dense(16,activation='swish')(x)
#     x=tf.keras.layers.Dense(16,activation='swish')(x)

    outputs = tf.keras.layers.Dense(classes, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]
    )
    return model

In [26]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = early, mode = 'max', restore_best_weights = True)
import keras
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
model = get_compiled_model()
model.summary(
    line_length=None,
    positions=None,
    print_fn=None,
    expand_nested=False,
    show_trainable=False,
    layer_range=None,
)
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

model_training_history = model.fit(x = train_data, y = one_hot_encoded_labels_train, epochs = epoch , batch_size = 20,
                                                   shuffle = True, validation_data=(test_data, one_hot_encoded_labels_test),
                                                   callbacks = [early_stopping_callback])



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None)  0           []                               
                                ]                                                                 
                                                                                                  
 frame_position_embedding (Posi  (None, None, 15)    450         ['input_1[0][0]']                
 tionalEmbedding)                                                                                 
                                                                                                  
 transformer_layer (Transformer  (None, None, 15)    33724       ['frame_position_embedding[0][0]'
 Encoder)                                                        ]                            

In [27]:
y_pred_train = model.predict(train_data)
y_pred_test = model.predict(test_data)

736/736 [==============================] - 3s 3ms/step


In [ ]:
def accuracymodel1(y_pred,train_label):
    SumCorrect = 0
    for i in range(y_pred.shape[0]):
        a = np.array(y_pred[i])
        a = a.argmax()
        if a==train_label[i,0]:
            SumCorrect = SumCorrect + 1
    return SumCorrect/y_pred.shape[0]

In [29]:
accuracymodel1(y_pred_train,train_label)
accuracymodel1(y_pred_test,test_label)

0.7827490970894413

In [30]:
def accuracymodel2(y_pred,train_label):
    SumCorrect = 0
    for i in range(y_pred.shape[0]):
        a = np.array(y_pred[i])
        b = np.array(y_pred[i])
        a = a.argmax()
        b[a] = 0
        b = b.argmax()
        
        if a==train_label[i,0] or b==train_label[i,0]:
            SumCorrect = SumCorrect + 1
    return SumCorrect/y_pred.shape[0]

In [31]:
accuracymodel2(y_pred_train,train_label)
accuracymodel2(y_pred_test,test_label)

0.9708944125770129

In [32]:
def accuracymodel3(y_pred,train_label):
    SumCorrect = 0
    for i in range(y_pred.shape[0]):
        a = np.array(y_pred[i])
        b = np.array(y_pred[i])
        c = np.array(y_pred[i])
        a = a.argmax()
        b[a] = 0
        b = b.argmax()
        c[a] = 0
        c[b] = 0
        c = c.argmax()
        
        if a==train_label[i,0] or b==train_label[i,0] or c==train_label[i,0]:
            SumCorrect = SumCorrect + 1
    return SumCorrect/y_pred.shape[0]

In [33]:
accuracymodel3(y_pred_train,train_label)
accuracymodel3(y_pred_test,test_label)

1.0

In [34]:
A = np.array([[accuracymodel1(y_pred_train,train_label), accuracymodel2(y_pred_train,train_label),accuracymodel3(y_pred_train,train_label)],
             [accuracymodel1(y_pred_test,test_label), accuracymodel2(y_pred_test,test_label),accuracymodel3(y_pred_test,test_label)]])
print(A)
Name = "Transformers-K"+str(K)+"-TTP"+str(TTP)+"-sequence" +str(sequence)
np.savetxt(r"/content/drive/MyDrive/Paper3/results"+"/"+Name+'.csv', A, delimiter=',')   # X is an array
model.save(r"/content/drive/MyDrive/Paper3/results"+"/"+Name+'.h5')  # creates a HDF5 file 'my_model.h5'

[[0.73427529 0.96153725 1.        ]
 [0.7827491  0.97089441 1.        ]]
